#### 단순한 케이스: 모든 특징의 타입이 같은 경우

In [1]:
import pandas as pd
import os

os.chdir(r"D:\강의 자료\2008_온라인 강의 머신러닝 성능 향상을 위한 데이터 탐색과 전처리 심화\데이터\실습 데이터")

In [2]:
df = pd.read_csv("cleveland.csv")

In [3]:
# 특징과 라벨 분리
X = df.drop('Output', axis = 1)
Y = df['Output']

In [4]:
# 학습 데이터와 평가 데이터로 분리
from sklearn.model_selection import train_test_split
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y)

In [5]:
# 결측치 확인
Train_X.isnull().sum()
# 결측치가 많지 않음
# 지워도 무방한 수치이지만, 새로 들어온 데이터에 결측이 있을 수도 있다는 도메인 지식이 있다고 가정

Age         0
Sex         0
Cp          0
Trestbps    0
Chol        0
Fbs         0
Restecg     0
Thalach     0
Exang       0
Oldpeak     0
Slope       0
Ca          2
Thal        1
dtype: int64

In [6]:
# 평균 상관 계수 확인 (주의: 모든 변수가 연속형이므로 가능한 접근)
Train_X.corr().sum() / (len(Train_X.columns) - 1)

# 수치가 높지 않다고 판단 => 특징 간 관계가 크지 않음 => 대표값 대체 활용 가능 판단

Age         0.188737
Sex         0.128624
Cp          0.082620
Trestbps    0.140691
Chol        0.130102
Fbs         0.114038
Restecg     0.155741
Thalach    -0.080396
Exang       0.171009
Oldpeak     0.221944
Slope       0.186604
Ca          0.144485
Thal        0.198240
dtype: float64

In [7]:
# 대표값을 활용한 결측치 대체
from sklearn.impute import SimpleImputer

# SimpleImputer 인스턴스화
SI = SimpleImputer(strategy = 'mean')

# 학습
SI.fit(Train_X)

# sklearn instance의 출력은 ndarray이므로 다시 DataFrame으로 바꿔줌
Train_X = pd.DataFrame(SI.transform(Train_X), columns = Train_X.columns)
Test_X = pd.DataFrame(SI.transform(Test_X), columns = Test_X.columns)

In [8]:
Train_X.isnull().sum()

Age         0
Sex         0
Cp          0
Trestbps    0
Chol        0
Fbs         0
Restecg     0
Thalach     0
Exang       0
Oldpeak     0
Slope       0
Ca          0
Thal        0
dtype: int64

#### 복잡한 케이스: 다른 타입의 특징이 있는 경우

In [9]:
df = pd.read_csv("saheart.csv")

In [10]:
# 특징과 라벨 분리
X = df.drop('Chd', axis = 1)
Y = df['Chd']

In [11]:
# 학습 데이터와 평가 데이터로 분리
from sklearn.model_selection import train_test_split
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y)

In [12]:
# 결측치 확인
Train_X.isnull().sum()
# 결측치가 많지 않음
# 지워도 무방한 수치이지만, 새로 들어온 데이터에 결측이 있을 수도 있다는 도메인 지식이 있다고 가정

Sbp          0
Tobacco      0
Ldl          0
Adiposity    0
Typea        0
Obesity      8
Alcohol      5
Age          0
Famhist      5
dtype: int64

In [13]:
# 평균 상관 계수 확인 (주의: 모든 변수가 연속형이므로 가능한 접근)
Train_X.corr().sum() / (len(Train_X.columns) - 1)

# 수치가 높지 않다고 판단 => 특징 간 관계가 크지 않음 => 대표값 대체 활용 가능 판단

Sbp          0.291340
Tobacco      0.207288
Ldl          0.308865
Adiposity    0.439088
Typea        0.105319
Obesity      0.380544
Alcohol      0.213968
Age          0.382141
Famhist      0.258030
dtype: float64

In [14]:
# Famhist: 범주형 변수
# 그 외 변수: 연속형 변수

# 대표값을 평균을 사용할지, 최빈값을 사용할지 결정이 어려움 => 둘 다 사용해야 함
# 따라서 데이터를 분할해야 함

Train_X_cate = Train_X[['Famhist']]
Train_X_cont = Train_X.drop('Famhist', axis = 1)

Test_X_cate = Test_X[['Famhist']]
Test_X_cont = Test_X.drop('Famhist', axis = 1)

In [15]:
# 대표값을 활용한 결측치 대체
from sklearn.impute import SimpleImputer

# SimpleImputer 인스턴스화
SI_mode = SimpleImputer(strategy = 'most_frequent')
SI_mean = SimpleImputer(strategy = 'mean')

# 학습
SI_mode.fit(Train_X_cate)
SI_mean.fit(Train_X_cont)

# sklearn instance의 출력은 ndarray이므로 다시 DataFrame으로 바꿔줌
Train_X_cate = pd.DataFrame(SI_mode.transform(Train_X_cate),
                            columns = Train_X_cate.columns)

Test_X_cate = pd.DataFrame(SI_mode.transform(Test_X_cate),
                           columns = Test_X_cate.columns)

Train_X_cont = pd.DataFrame(SI_mean.transform(Train_X_cont),
                            columns = Train_X_cont.columns)

Test_X_cont = pd.DataFrame(SI_mean.transform(Test_X_cont),
                           columns = Test_X_cont.columns)

# 다시 두 데이터를 이어붙여야 함
Train_X = pd.concat([Train_X_cate, Train_X_cont], axis = 1)
Test_X = pd.concat([Test_X_cate, Test_X_cont], axis = 1)

In [16]:
Train_X.isnull().sum()

Famhist      0
Sbp          0
Tobacco      0
Ldl          0
Adiposity    0
Typea        0
Obesity      0
Alcohol      0
Age          0
dtype: int64

In [17]:
# Tip. 이진형 변수와 연속형 변수만 포함된 경우에는 SI_mean만 사용하여 결측치를 평균으로 대체한 뒤에, 
# 이진형 변수에 대해서만 round 처리를 하면 하나의 인스턴스만 활용할 수 있음